## Preparing the Environment

In [20]:
from __future__ import print_function
import numpy as np

from ifqi import envs
from ifqi.evaluation import evaluation
from ifqi.algorithms.fqi.FQI import FQI

from scipy.optimize import curve_fit

In [22]:
mdp = envs.LQG1D()
state_dim, action_dim, reward_dim = envs.get_space_info(mdp)
reward_idx = state_dim + action_dim
discrete_actions = np.array([-8, -7, -6, -5, -4, -3, -2.5, -2, -1.5, -1, -.75,
                             -.5, -.25, 0, .25, .5, .75, 1, 1.5, 2, 2.5, 3, 4,
                             5, 6, 7, 8])
dataset = evaluation.collect_episodes(mdp, n_episodes=1000)
sast = np.append(dataset[:, :reward_idx],
                 dataset[:, reward_idx + reward_dim:-1],
                 axis=1)
r = dataset[:, reward_idx]

## Learning with FQI and `curve_fit`

In [56]:
class Regressor:
    __slots__ = ('b', 'k')
    
    def __init__(self, b=0, k=0):
        self.b = b
        self.k = k
    
    def fit(self, X, y):
        #print("fitting: ", end='')
        (self.b, self.k), pcov = curve_fit(self.Q, X, y)
        #print(self.b, self.k)
    
    def Q(self, sa, b, k):
        return b - (sa[:, 1] - k *  sa[:, 0]) ** 2
    
    def predict(self, X):
        return self.Q(X, self.b, self.k)

regressor = Regressor()

In [57]:
fqi = FQI(estimator=regressor,
          state_dim=state_dim,
          action_dim=action_dim,
          discrete_actions=discrete_actions,
          gamma=mdp.gamma,
          horizon=mdp.horizon,
          scaled=False,
          features=None,
          verbose=False)

In [58]:
fqi.fit(sast, r)

In [59]:
initial_states = np.array([[1, 1]]).T

In [61]:
values = evaluation.evaluate_policy(mdp, fqi, initial_states=initial_states)
print(values)

(-9015855.9401270039, 12382.475403956161, 100.0, 0.0)


## Trying to evaluate Optimal Policy

In [52]:
K = mdp.computeOptimalK()[0][0]
K

-0.61525124566301148

In [51]:
class OptimalPolicy:
    
    def draw_action(self, states, absorbing, evaluation=False):
        print("states: {} action: {}".format(states, K*states))
        return K*states
    
optimalP = GreedyPolicy()


In [54]:
values = evaluation.evaluate_policy(mdp, greedyP, initial_states=initial_states)
print(values)

states: [1] action: [-0.61525125]
states: [ 0.12029013] action: [-0.07400865]
states: [ 0.13699011] action: [-0.08428333]
states: [-0.19216612] action: [ 0.11823045]
states: [-0.09065101] action: [ 0.05577315]
states: [-0.00419045] action: [ 0.00257818]
states: [ 0.02360065] action: [-0.01452033]
states: [-0.03120313] action: [ 0.01919776]
states: [-0.06247402] action: [ 0.03843722]
states: [-0.13703919] action: [ 0.08431353]
states: [ 0.05855304] action: [-0.03602483]
states: [ 0.03248986] action: [-0.01998943]
states: [ 0.08308602] action: [-0.05111878]
states: [ 0.14233431] action: [-0.08757136]
states: [-0.00937366] action: [ 0.00576716]
states: [ 0.0273491] action: [-0.01682657]
states: [ 0.05986723] action: [-0.03683339]
states: [-0.07149178] action: [ 0.0439854]
states: [-0.1099668] action: [ 0.06765721]
states: [-0.00808528] action: [ 0.00497448]
states: [ 0.032561] action: [-0.02003319]
states: [-0.00997175] action: [ 0.00613513]
states: [ 0.03884567] action: [-0.02389985]
sta